In [17]:
import re
import numpy as np
import pandas as pd

# --- helpers ---
VAL_PAT = re.compile(r"\s*([+-]?\d+(?:\.\d+)?)\s*[±\+-\-~]*\s*([+-]?\d+(?:\.\d+)?)\s*")

def fmt(mean, ci):
    return f"{float(mean):.4f} \\pm {float(ci):.4f}"

def parse_value_str(s):
    if isinstance(s, (int, float)):  # mean only
        return float(s), np.nan
    if not isinstance(s, str):
        return np.nan, np.nan
    s = s.replace("+-", "±").replace("+/-", "±").replace("±", " ± ")
    m = VAL_PAT.match(s.strip())
    return (float(m.group(1)), float(m.group(2))) if m else (np.nan, np.nan)

def split_method_metric(metric_str):
    # Examples in your baseline:
    # 'DR-kNN MSE' -> ('DR-kNN','MSE')
    # 'DR-kNN DR-value' -> ('DR-kNN','Value')
    # 'kNN MSE' -> ('kNN','MSE')
    # 'kNN plug-value' -> ('kNN','Value')
    # 'MSM weighted contrast' -> ('MSM','Value')
    # 'Seq-DR value' -> ('Seq-DR','Value')
    s = str(metric_str).strip()
    s = s.replace("plug-value", "Value").replace("DR-value", "Value").replace("weighted contrast","Value")
    parts = s.split()
    method = " ".join(parts[:-1]) if len(parts) > 1 else parts[0]
    metric = parts[-1].capitalize() if len(parts) > 1 else "Value"
    if metric not in ("MSE","Value"):
        metric = "Value"
    return method, metric

# --- normalize ours ---
def normalize_ours(df_ours, method_name="C-kNN-LSH (ours)"):
    need = {"action","mse_mean","mse_ci95","dr_value_mean","dr_value_ci95"}
    missing = need - set(df_ours.columns)
    if missing:
        raise ValueError(f"df_ours missing columns: {missing}")
    recs = []
    for r in df_ours.itertuples(index=False):
        action = r.action
        recs.append({"action": action, "method": method_name, "metric": "MSE",
                     "mean": float(r.mse_mean), "ci95": float(r.mse_ci95)})
        recs.append({"action": action, "method": method_name, "metric": "Value",
                     "mean": float(r.dr_value_mean), "ci95": float(r.dr_value_ci95)})
    out = pd.DataFrame(recs)
    out["cell"] = out.apply(lambda x: fmt(x["mean"], x["ci95"]), axis=1)
    return out

# --- normalize baselines ---
def normalize_baselines(df_base):
    need = {"action","metric","value"}
    missing = need - set(df_base.columns)
    if missing:
        raise ValueError(f"df_base missing columns: {missing}")
    rows = []
    for r in df_base.itertuples(index=False):
        mth, met = split_method_metric(r.metric)
        mean, ci = parse_value_str(r.value)
        rows.append({"action": r.action, "method": mth, "metric": met,
                     "mean": mean, "ci95": ci})
    out = pd.DataFrame(rows)
    out["cell"] = out.apply(lambda x: fmt(x["mean"], x["ci95"]) if pd.notna(x["mean"]) else "", axis=1)
    return out

# --- build paper table ---
import pandas as pd
import numpy as np

def build_paper_table(df_ours, df_base,
                      visits_train=53270.2, visits_test=13432.8,
                      methods=("C-kNN-LSH (ours)","DR-kNN","kNN","T-Learner","MSM","Seq-DR"),
                      action_order=("A_dose1","A_dose2","A_dose3","A_dose4","A_vax_any")):
    # ---- normalize_ours / normalize_baselines defined earlier ----
    ours = normalize_ours(df_ours)
    base = normalize_baselines(df_base)
    alln = pd.concat([ours, base], ignore_index=True)

    # pivot to (method, metric)
    cols_full = pd.MultiIndex.from_product([methods, ["MSE","Value"]])
    piv = (alln.pivot_table(index="action", columns=["method","metric"], values="cell", aggfunc="first")
                .reindex(columns=cols_full))

    # visits as strings (avoid floats later)
    piv[("Visits","Train (avg)")] = f"{visits_train:,.1f}"
    piv[("Visits","Test (avg)")]  = f"{visits_test:,.1f}"

    # order columns
    ordered_cols = []
    for m in methods:
        ordered_cols.extend([(m,"MSE"),(m,"Value")])
    ordered_cols.extend([("Visits","Train (avg)"),("Visits","Test (avg)")])
    piv = piv.reindex(columns=pd.MultiIndex.from_tuples(ordered_cols))

    # order actions
    order = [a for a in action_order if a in piv.index]
    order += [a for a in piv.index if a not in order]
    piv = piv.loc[order]

    # IMPORTANT: clear MultiIndex names properly (list-like for both levels)
    if isinstance(piv.columns, pd.MultiIndex) and piv.columns.nlevels == 2:
        piv.columns = pd.MultiIndex.from_tuples(piv.columns.tolist(), names=[None, None])
    piv.index.name = None
    return piv


def to_latex_block(piv):
    methods = ["C-kNN-LSH (ours)","DR-kNN","kNN","T-Learner","MSM","Seq-DR"]

    def safe_str(x):
        if x is None or (isinstance(x, float) and np.isnan(x)):
            return ""
        return x if isinstance(x, str) else f"{x}"

    header = [
        r"\begin{table}[t]",
        r"\centering",
        r"\caption{Metrics by action and method. ``Value'' denotes DR-value for C-kNN-LSH and DR-kNN, plug-in policy value for kNN and T-Learner, weighted contrast for MSM, and sequential DR value for Seq-DR. MSE is only defined for C-kNN-LSH, DR-kNN, kNN, and T-Learner.}",
        r"\label{tab:methods-by-action-full}",
        r"\resizebox{\linewidth}{!}{%",
        r"\begin{tabular}{l " + " ".join(["cc"]*len(methods)) + " cc}",
        r"\toprule",
        r"\multirow{2}{*}{\textbf{Action}} & " +
        " & ".join([fr"\multicolumn{{2}}{{c}}{{\textbf{{{m}}}}}" for m in methods]) +
        r" & \multicolumn{2}{c}{\textbf{Visits}} \\",
        r"& " + " & ".join([r"\textbf{MSE} & \textbf{Value}"]*len(methods)) +
        r" & \textbf{Train (avg)} & \textbf{Test (avg)} \\",
        r"\midrule"
    ]

    lines = []
    for action, row in piv.iterrows():
        cells = [action.replace("_", r"\_")]
        for m in methods:
            # pull safely and stringify
            mse  = row.get((m,"MSE"),  np.nan)
            val  = row.get((m,"Value"), np.nan)
            cells.append(safe_str(mse))
            cells.append(safe_str(val))
        cells.append(safe_str(row.get(("Visits","Train (avg)"))))
        cells.append(safe_str(row.get(("Visits","Test (avg)"))))
        lines.append(" & ".join(map(safe_str, cells)) + r" \\")
    footer = [r"\bottomrule", r"\end{tabular}%", r"}", r"\end{table}"]
    return "\n".join(header + lines + footer)


# --------- USE IT ----------
df_ours = df
df_base = b
# Build table and export LaTeX:
piv = build_paper_table(df_ours, df_base)
tex = to_latex_block(piv)
print(tex)
with open("methods_by_action_full.tex","w") as f:
    f.write(tex)



\begin{table}[t]
\centering
\caption{Metrics by action and method. ``Value'' denotes DR-value for C-kNN-LSH and DR-kNN, plug-in policy value for kNN and T-Learner, weighted contrast for MSM, and sequential DR value for Seq-DR. MSE is only defined for C-kNN-LSH, DR-kNN, kNN, and T-Learner.}
\label{tab:methods-by-action-full}
\resizebox{\linewidth}{!}{%
\begin{tabular}{l cc cc cc cc cc cc cc}
\toprule
\multirow{2}{*}{\textbf{Action}} & \multicolumn{2}{c}{\textbf{C-kNN-LSH (ours)}} & \multicolumn{2}{c}{\textbf{DR-kNN}} & \multicolumn{2}{c}{\textbf{kNN}} & \multicolumn{2}{c}{\textbf{T-Learner}} & \multicolumn{2}{c}{\textbf{MSM}} & \multicolumn{2}{c}{\textbf{Seq-DR}} & \multicolumn{2}{c}{\textbf{Visits}} \\
& \textbf{MSE} & \textbf{Value} & \textbf{MSE} & \textbf{Value} & \textbf{MSE} & \textbf{Value} & \textbf{MSE} & \textbf{Value} & \textbf{MSE} & \textbf{Value} & \textbf{MSE} & \textbf{Value} & \textbf{Train (avg)} & \textbf{Test (avg)} \\
\midrule
A\_dose1 & 1.0429 \pm 0.0782 & 0.0571 \

In [11]:
b = pd.read_csv('all_methods_summary.csv')
b.head()


,action,metric,value
0,A_dose1,DR-kNN MSE,2.5023 ± 0.1068
1,A_dose1,DR-kNN DR-value,0.0604 ± 0.0781
2,A_dose1,kNN MSE,2.5086 ± 0.0949
3,A_dose1,kNN plug-value,0.0670 ± 0.0776
4,A_dose1,T-Learner MSE,2.4629 ± 0.0955


In [10]:
df = pd.read_csv('cknn_vax_actions_summary_nodupp.csv')
df.head()

,action,mse_mean,mse_ci95,dr_value_mean,dr_value_ci95
0,A_dose1,1.042907,0.078177,0.057056,0.020515
1,A_dose2,1.037030,0.078574,0.052650,0.011001
2,A_dose3,1.041522,0.081021,0.056194,0.018298
3,A_dose4,1.036988,0.078486,0.400109,0.154616
4,A_vax_any,1.038675,0.077883,0.330241,0.128906


In [1]:
import math, random
import numpy as np

# =============== Utilities ===============

def add_bias(X):
    return np.concatenate([X, np.ones((X.shape[0],1), dtype=X.dtype)], axis=1)

def ridge_regression_fit(X, y, l2=1e-2):
    XT = X.T
    A = XT @ X
    I = np.eye(A.shape[0], dtype=X.dtype)
    I[-1, -1] = 0.0  # don't regularize bias
    w = np.linalg.solve(A + l2*I, XT @ y)
    return w

def ridge_regression_predict(W, X):
    return X @ W

def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def logistic_regression_fit(X, y, l2=1e-3, lr=0.1, epochs=300, batch=2048, seed=1337):
    rng = np.random.default_rng(seed)
    N, D = X.shape
    w = rng.normal(0, 0.01, size=(D,)).astype(np.float32)
    y = y.astype(np.float32)
    for ep in range(epochs):
        perm = rng.permutation(N)
        for s in range(0, N, batch):
            e = min(N, s+batch)
            xb = X[perm[s:e]]; yb = y[perm[s:e]]
            p = sigmoid(xb @ w)
            # gradient (+ L2 on non-bias)
            grad = (xb.T @ (p - yb)) / (e - s)
            grad[:-1] += l2 * w[:-1]
            w -= lr * grad
    return w

def logistic_regression_predict_proba(W, X):
    return sigmoid(X @ W)

def flatten_time(phi, A, Y):
    """phi: (n,T,d) object -> (N,d), A,Y -> (N,)"""
    n = len(phi)
    ds = [p.shape[0] for p in phi]
    d = phi[0].shape[1]
    Ntot = sum(ds)
    out_phi = np.zeros((Ntot, d), dtype=np.float32)
    out_A = np.zeros((Ntot,), dtype=np.int64)
    out_Y = np.zeros((Ntot,), dtype=np.float32)
    idx = 0
    for i in range(n):
        T = phi[i].shape[0]
        out_phi[idx:idx+T] = phi[i]
        out_A[idx:idx+T] = A[i]
        out_Y[idx:idx+T] = Y[i]
        idx += T
    return out_phi, out_A, out_Y

# =============== History features (no torch) ===============

def build_history_features(X_obj, A_obj, Y_obj, d_latent=16, seed=1337, alpha=0.5):
    """
    X_obj[i]: (T_i, d_x) features per visit *before* treatment
    Returns Phi[i]: (T_i, d_latent)
    Idea: concat [x_t, a_{t-1}, y_{t-1}, EMA(x_{<t}), cum-mean(y_{<t})] -> random projection + tanh
    """
    rng = np.random.default_rng(seed)
    N = len(X_obj)
    d_x = X_obj[0].shape[1]

    d_raw = d_x + 1 + 1 + d_x + 1
    W = rng.normal(0, 1/np.sqrt(d_raw), size=(d_raw, d_latent)).astype(np.float32)
    b = rng.normal(0, 0.1, size=(d_latent,)).astype(np.float32)

    Phi = []
    for i in range(N):
        Xi = X_obj[i]; Ai = A_obj[i]; Yi = Y_obj[i]
        T = Xi.shape[0]
        phi_i = np.zeros((T, d_latent), dtype=np.float32)
        x_ema = np.zeros((d_x,), dtype=np.float32)
        y_csum = 0.0
        for t in range(T):
            x_t = Xi[t]
            a_prev = Ai[t-1] if t>0 else 0.0
            y_prev = Yi[t-1] if t>0 else 0.0
            if t>0:
                x_ema = alpha * x_ema + (1-alpha) * Xi[t-1]
                y_csum += Yi[t-1]
                y_cmean = y_csum / t
            else:
                x_ema = np.zeros_like(x_t)
                y_cmean = 0.0
            raw = np.concatenate([x_t, [a_prev], [y_prev], x_ema, [y_cmean]]).astype(np.float32)
            phi_i[t] = np.tanh(raw @ W + b)
        Phi.append(phi_i)
    return np.array(Phi, dtype=object)

# =============== ANN (HNSW or exact kNN fallback) ===============

try:
    import hnswlib
    class ANNIndex:
        def __init__(self, dim, space="l2", ef=100, M=32):
            self.index = hnswlib.Index(space=space, dim=dim)
            self.ef = ef; self.M = M; self.built = False
        def build(self, X, ids=None):
            if ids is None: ids = np.arange(X.shape[0])
            self.index.init_index(max_elements=X.shape[0], ef_construction=self.ef, M=self.M)
            self.index.add_items(X.astype(np.float32), ids.astype(np.int64))
            self.index.set_ef(self.ef); self.built = True
        def query(self, Q, k=50):
            return self.index.knn_query(Q.astype(np.float32), k=k)
except Exception:
    # exact kNN fallback (no dependency)
    class ANNIndex:
        def __init__(self, dim, space="l2", **kwargs):
            self.X = None
        def build(self, X, ids=None):
            self.X = X.astype(np.float32)
        def query(self, Q, k=50):
            # squared l2
            d2 = ((Q[:,None,:] - self.X[None,:,:])**2).sum(axis=2)
            idx = np.argsort(d2, axis=1)[:, :k]
            d  = np.take_along_axis(d2, idx, axis=1)
            return idx, d

# =============== DR-kNN estimator ===============

def dr_knn_mu(phi_tr, A_tr, Y_tr, query_phi, a_query, out_W, prop_W, k=50, temp=1.0):
    """
    Double-robust kNN: neighbors on phi among train rows with A==a_query.
    out_W: ridge on [phi, onehot(a)](+bias)
    prop_W: logistic on phi(+bias)
    """
    mask_a = (A_tr == a_query)
    phi_a = phi_tr[mask_a]
    ann = ANNIndex(dim=phi_tr.shape[1])
    ann.build(phi_a)

    def design_outcome(phi, a_vec):
        a0 = (a_vec==0).astype(np.float32).reshape(-1,1)
        a1 = (a_vec==1).astype(np.float32).reshape(-1,1)
        Z = np.concatenate([phi, a0, a1], axis=1)
        return add_bias(Z)

    def m_pred(phi, a_scalar):
        Z = design_outcome(phi, np.full((phi.shape[0],), a_scalar, dtype=np.int64))
        return ridge_regression_predict(out_W, Z)

    def e_pred(phi):
        return logistic_regression_predict_proba(prop_W, add_bias(phi))

    labels, dists = ann.query(query_phi, k=min(k, len(phi_a)))
    Wker = np.exp(-dists / max(temp, 1e-6))
    Wker /= (Wker.sum(axis=1, keepdims=True) + 1e-8)

    # train-side predictions once
    e_train = e_pred(phi_tr)
    e_train = e_train if a_query==1 else (1.0 - e_train)
    m_train_a = m_pred(phi_tr, a_query)

    mu = np.zeros(query_phi.shape[0], dtype=np.float32)
    # indices of global train rows corresponding to A==a_query
    idx_map = np.where(mask_a)[0]
    for i in range(query_phi.shape[0]):
        local = labels[i]
        global_idx = idx_map[local]
        w = Wker[i]

        res = Y_tr[global_idx] - m_train_a[global_idx]
        e_neighbors = e_train[global_idx]
        iw = 1.0 / np.clip(e_neighbors, 1e-3, 1-1e-3)
        w_tilde = w * iw; w_tilde /= (w_tilde.sum() + 1e-8)

        m_q = m_pred(query_phi[i:i+1], a_query)[0]
        mu[i] = m_q + np.sum(w_tilde * res)
    return mu

# =============== Runner: pass your lists here ===============

def run_cknn_numpy(X_list, A_list, Y_list, d_latent=16, seed=1337, k=50, temp=1.0):
    rng = np.random.default_rng(seed)
    n = len(X_list)
    idx = np.arange(n); rng.shuffle(idx)
    n_te = max(1, int(0.2*n))
    te_idx, tr_idx = idx[:n_te], idx[n_te:]

    # object arrays (ragged ok)
    X_obj = np.array([np.asarray(x, dtype=np.float32) for x in X_list], dtype=object)
    A_obj = np.array([np.asarray(a, dtype=np.int64) for a in A_list], dtype=object)
    Y_obj = np.array([np.asarray(y, dtype=np.float32) for y in Y_list], dtype=object)

    # history embeddings
    Phi = build_history_features(X_obj, A_obj, Y_obj, d_latent=d_latent, seed=seed)
    Phi_tr = Phi[tr_idx]; A_tr = A_obj[tr_idx]; Y_tr = Y_obj[tr_idx]
    Phi_te = Phi[te_idx]; A_te = A_obj[te_idx]; Y_te = Y_obj[te_idx]

    # flatten to visit-level
    phi_tr_flat, A_tr_flat, Y_tr_flat = flatten_time(Phi_tr, A_tr, Y_tr)
    phi_te_flat, A_te_flat, Y_te_flat = flatten_time(Phi_te, A_te, Y_te)

    # fit outcome model: y ~ [phi, onehot(a)] + bias
    a0 = (A_tr_flat==0).astype(np.float32).reshape(-1,1)
    a1 = (A_tr_flat==1).astype(np.float32).reshape(-1,1)
    Z_out_tr = np.concatenate([phi_tr_flat, a0, a1], axis=1)
    out_W = ridge_regression_fit(add_bias(Z_out_tr), Y_tr_flat, l2=1e-2)

    # fit propensity: a ~ phi + bias
    prop_W = logistic_regression_fit(add_bias(phi_tr_flat), A_tr_flat.astype(np.float32),
                                     l2=1e-3, lr=0.1, epochs=300, batch=2048, seed=seed)

    # DR-kNN mu(a|H) on test
    mu0 = dr_knn_mu(phi_tr_flat, A_tr_flat, Y_tr_flat, phi_te_flat, a_query=0,
                    out_W=out_W, prop_W=prop_W, k=k, temp=temp)
    mu1 = dr_knn_mu(phi_tr_flat, A_tr_flat, Y_tr_flat, phi_te_flat, a_query=1,
                    out_W=out_W, prop_W=prop_W, k=k, temp=temp)

    # evaluation proxy: observed-treatment MSE
    y_pred_obs = np.where(A_te_flat==1, mu1, mu0)
    mse = float(np.mean((Y_te_flat - y_pred_obs)**2))

    # simple policy: treat if mu1 > mu0, DR value estimate
    policy = (mu1 > mu0).astype(np.int64)
    # propensity for observed A on test
    e_te = logistic_regression_predict_proba(prop_W, add_bias(phi_te_flat))
    e_te = np.where(A_te_flat==1, e_te, 1.0 - e_te)
    # m_obs (outcome regression at observed A)
    a0_te = (A_te_flat==0).astype(np.float32).reshape(-1,1)
    a1_te = (A_te_flat==1).astype(np.float32).reshape(-1,1)
    Z_out_te = np.concatenate([phi_te_flat, a0_te, a1_te], axis=1)
    m_obs = ridge_regression_predict(out_W, add_bias(Z_out_te))
    dr_value = float(np.mean(((policy==A_te_flat)/np.clip(e_te,1e-3,1-1e-3)) * (Y_te_flat - m_obs)
                             + (policy*mu1 + (1-policy)*mu0)))

    return {
        "mse": mse,
        "dr_value": dr_value,
        "mu0": mu0, "mu1": mu1,
        "A_te": A_te_flat, "Y_te": Y_te_flat
    }
def knn_mu(phi_tr, A_tr, Y_tr, query_phi, a_query, k=50, temp=1.0):
    mask = (A_tr==a_query)
    X = phi_tr[mask]; y = Y_tr[mask]
    # exact kNN if no hnswlib
    d2 = ((query_phi[:,None,:]-X[None,:,:])**2).sum(2)
    idx = np.argsort(d2,1)[:, :min(k, len(X))]
    d  = np.take_along_axis(d2, idx, axis=1)
    W = np.exp(-d/ max(temp,1e-6)); W /= (W.sum(1,keepdims=True)+1e-8)
    return (W * y[idx]).sum(1)
def tlearner():
    # train
    Z0, y0 = phi_tr_flat[A_tr_flat==0], Y_tr_flat[A_tr_flat==0]
    Z1, y1 = phi_tr_flat[A_tr_flat==1], Y_tr_flat[A_tr_flat==1]
    W0 = ridge_regression_fit(add_bias(Z0), y0, l2=1e-2)
    W1 = ridge_regression_fit(add_bias(Z1), y1, l2=1e-2)
    # predict potential outcomes on test
    mu0 = ridge_regression_predict(W0, add_bias(phi_te_flat))
    mu1 = ridge_regression_predict(W1, add_bias(phi_te_flat))
    # compute MSE & DR value as before
def msm():
    e = logistic_regression_predict_proba(prop_W, add_bias(phi_te_flat))
    e_obs = np.where(A_te_flat==1, e, 1-e)
    # simple (non-sequential) stabilized weight per visit
    pA = A_te_flat.mean()  # crude marginal; or per-time-bin marginal
    num = np.where(A_te_flat==1, pA, 1-pA)
    w = num / np.clip(e_obs,1e-3,1-1e-3)
    # weighted outcome difference
    tau_msm = np.average(Y_te_flat, weights=w*(A_te_flat==1)) - np.average(Y_te_flat, weights=w*(A_te_flat==0))
# ================== Example call ==================
# results = run_cknn_numpy(X_list, A_list, Y_list, d_latent=16, seed=1337, k=50, temp=1.0)
# print("Observed-treatment MSE:", results["mse"])
# print("DR value (treat if mu1>mu0):", results["dr_value"])
# seeds = [1,2,3,4,5,6,7,8,9,10]
# rows = []
# for s in seeds:
#     res = run_cknn_numpy(X_list, A_list, Y_list, d_latent=16, seed=s, k=50, temp=1.0)
#     rows.append((s, res["mse"], res["dr_value"]))
#     arr = np.array([[r[1], r[2]] for r in rows], dtype=float)
#     m_mse, m_dr = arr.mean(0)
#     s_mse, s_dr = arr.std(0, ddof=1)
#     print(f"MSE: {m_mse:.4f} ± {1.96*s_mse/np.sqrt(len(seeds)):.4f}")
#     print(f"DR value: {m_dr:.4f} ± {1.96*s_dr/np.sqrt(len(seeds)):.4f}")


In [2]:
import numpy as np
import pandas as pd
def asof_per_patient(left_df, right_df, id_col, left_time, right_time, outcome_col,direction="forward", allow_exact_matches=True):
    out = []
    for pid, L in left_df.groupby(id_col, sort=False):
        R = right_df[right_df[id_col] == pid]
        if L.empty:
            continue
        # sort & drop NaT within this patient
        L = L.copy(); R = R.copy()
        L[left_time] = pd.to_datetime(L[left_time], errors="coerce").dt.tz_localize(None)
        R[right_time] = pd.to_datetime(R[right_time], errors="coerce").dt.tz_localize(None)
        L = L[L[left_time].notna()].sort_values(left_time)
        R = R[R[right_time].notna()].sort_values(right_time)
        if R.empty:
            # still add rows with _y_next = NaN to keep alignment if you need it later
            tmp = L.copy()
            tmp["_y_next"] = np.nan
            tmp["_y_next_date"] = pd.NaT
        else:
            tmp = pd.merge_asof(
                L,
                R[[id_col, right_time, outcome_col]],
                left_on=left_time,
                right_on=right_time,
                direction=direction,
                allow_exact_matches=allow_exact_matches,
            ).rename(columns={outcome_col: "_y_next", right_time: "_y_next_date"})
        out.append(tmp)
    return pd.concat(out, axis=0, ignore_index=True)


def _clean_for_asof(df, id_col, time_col):
    df = df.copy()
    # parse to timezone-naive datetimes
    df[time_col] = pd.to_datetime(df[time_col], errors='coerce').dt.tz_localize(None)
    # drop rows with missing id or time (asof requires non-null)
    df = df[df[id_col].notna() & df[time_col].notna()]
    # sort by id then time (required for asof with "by")
    df = df.sort_values([id_col, time_col])
    # optional: drop duplicate rows on (id,time) if they exist
    df = df.drop_duplicates(subset=[id_col, time_col])
    return df
def build_triplets_from_XA_and_outcome(
    X_visit, outcome_df,
    id_col="PARTICIPANT_ID",
    visit_col="VISIT_START_DATE",
    A_col="A_booster",
    outcome_col="pasc_score",
    outcome_date_col="date",
    outcome_mode="next_value",
    max_forward_days=None
):
    dfX = _clean_for_asof(X_visit, id_col, visit_col)
    dfY = _clean_for_asof(outcome_df, id_col, outcome_date_col)

    # feature columns (exclude id/date/A and non-numeric)
    exclude = {id_col, visit_col, A_col}
    non_numeric = set(dfX.select_dtypes(include=["object"]).columns)
    X_cols = [c for c in dfX.columns if c not in exclude and c not in non_numeric]

    # NEXT outcome after each visit (no leakage)
    # out_next = (
    #     pd.merge_asof(
    #         dfX[[id_col, visit_col]],
    #         dfY[[id_col, outcome_date_col, outcome_col]],
    #         by=id_col,
    #         left_on=visit_col,
    #         right_on=outcome_date_col,
    #         direction="forward",
    #         allow_exact_matches=True,
    #     )
    #     .rename(columns={outcome_col: "_y_next", outcome_date_col: "_y_next_date"})
    # )
    # Use it like:
    left  = dfX[[id_col, visit_col]].copy()
    right = dfY[[id_col, outcome_date_col, outcome_col]].copy()
    out_next = asof_per_patient(
        left_df=left, right_df=right,
        id_col=id_col, left_time=visit_col, right_time=outcome_date_col,outcome_col=outcome_col,
        direction="forward", allow_exact_matches=True
    ).rename(columns={outcome_col: "_y_next", outcome_date_col: "_y_next_date"})
    


    if max_forward_days is not None:
        dt = (out_next["_y_next_date"] - dfX[visit_col]).dt.days
        out_next.loc[dt > max_forward_days, "_y_next"] = np.nan

    if outcome_mode == "delta_next":
        left  = dfX[[id_col, visit_col]].copy()
        right = dfY[[id_col, outcome_date_col, outcome_col]].copy()
        # out_prev = asof_per_patient(
        #     left_df=left, right_df=right,
        #     id_col=id_col, left_time=visit_col, right_time=outcome_date_col,outcome_col=outcome_col,
        #     direction="backward", allow_exact_matches=True
        # ).rename(columns={outcome_col: "_y_prev"})       
        # backward (previous outcome)
        out_prev = asof_per_patient(
            left_df=left, right_df=right,
            id_col=id_col, left_time=visit_col, right_time=outcome_date_col, outcome_col=outcome_col,
            direction="backward", allow_exact_matches=True
        )
        # <-- rename the columns that asof_per_patient created
        out_prev = out_prev.rename(columns={"_y_next": "_y_prev", "_y_next_date": "_y_prev_date"})
        
        # now this works:
        aligned = pd.concat([dfX[[id_col, visit_col]], out_next["_y_next"], out_prev["_y_prev"]], axis=1)
        aligned["Y_t"] = aligned["_y_next"] - aligned["_y_prev"]
        # out_prev = (
        #     pd.merge_asof(
        #         dfX[[id_col, visit_col]],
        #         dfY[[id_col, outcome_date_col, outcome_col]],
        #         by=id_col,
        #         left_on=visit_col,
        #         right_on=outcome_date_col,
        #         direction="backward",
        #         allow_exact_matches=True,
        #     ).rename(columns={outcome_col: "_y_prev"})
        # )
        # aligned = pd.concat([dfX[[id_col, visit_col]], out_next["_y_next"], out_prev["_y_prev"]], axis=1)
        # aligned["Y_t"] = aligned["_y_next"] - aligned["_y_prev"]
    elif outcome_mode == "next_value":
        aligned = pd.concat([dfX[[id_col, visit_col]], out_next["_y_next"]], axis=1)
        aligned["Y_t"] = aligned["_y_next"]
    else:
        raise ValueError("outcome_mode must be 'next_value' or 'delta_next'.")

    # Keep visits with a follow-up outcome
    keep = ~aligned["Y_t"].isna()
    dfX_kept = dfX.loc[keep].copy()
    aligned = aligned.loc[keep].copy()

    # Collect ragged sequences
    X_list, A_list, Y_list = [], [], []
    for pid, gX in dfX_kept.groupby(id_col):
        gX = gX.sort_values(visit_col)
        X_arr = gX[X_cols].to_numpy(dtype="float32")
        A_arr = gX[A_col].astype("int8").to_numpy()
        Y_arr = aligned.loc[gX.index, "Y_t"].astype("float32").to_numpy()
        if len(X_arr) == 0: 
            continue
        X_list.append(X_arr); A_list.append(A_arr); Y_list.append(Y_arr)

    return np.array(X_list, dtype=object), np.array(A_list, dtype=object), np.array(Y_list, dtype=object), X_cols

In [5]:
print("hello")

hello


In [ ]:
import numpy as np, pandas as pd
import time
ACTIONS = ["A_dose1","A_dose2","A_dose3","A_dose4","A_vax_any"]
SEEDS   = [1,2,3,4,5]#,6,7,8,9,10]   # shrink to e.g. [1,2,3,4,5] for speed
ID_COL, VISIT_COL = "PARTICIPANT_ID", "VISIT_START_DATE"
OUTCOME_NAME, OUTCOME_DATE, OUTCOME_MODE, MAX_FORWARD_DAYS = "pasc_score","date","delta_next",120

def zscore_Y(Y_list):
    all_y = np.concatenate([y for y in Y_list]) if len(Y_list) else np.array([0.])
    mu, sd = all_y.mean(), all_y.std() or 1.0
    return np.array([(y - mu)/sd for y in Y_list], dtype=object)

# 1) Precompute (X,A,Y) ONCE per action (this is the big time saver)
triplet_cache = {}
for A_col in ACTIONS:
    start_time = time.time()
    print(A_col)
    X_list, A_list, Y_list, _ = build_triplets_from_XA_and_outcome(
        X_visit=X_visit_with_A, outcome_df=pasc_df,
        id_col=ID_COL, visit_col=VISIT_COL,
        A_col=A_col, outcome_col=OUTCOME_NAME, outcome_date_col=OUTCOME_DATE,
        outcome_mode=OUTCOME_MODE, max_forward_days=MAX_FORWARD_DAYS
    )
    # optional standardization (consistent across seeds)
    Y_list_std = zscore_Y(Y_list)
    triplet_cache[A_col] = (X_list, A_list, Y_list_std)
    end_time = time.time()
    print(end_time - start_time)
# 2) Run seeds WITHOUT rebuilding triplets
rows = []
for A_col in ACTIONS:
    print("hello ", A_col)
    start_time = time.time()
    X_list, A_list, Y_list_std = triplet_cache[A_col]
    for s in SEEDS:
        res = run_cknn_numpy(X_list, A_list, Y_list_std, d_latent=16, seed=s, k=50, temp=1.0)
        rows.append({"action": A_col, "seed": s, "mse": res["mse"], "dr_value": res["dr_value"]})
    end_time = time.time()
    print("run  ", end_time - start_time)
df = pd.DataFrame(rows)

# 3) Summarize mean ± 95% CI
def summarize(group, col):
    mean = group[col].mean()
    sem  = group[col].std(ddof=1) / max(1, np.sqrt(len(group)))
    ci95 = 1.96 * sem
    return pd.Series({f"{col}_mean": mean, f"{col}_ci95": ci95})

summary = (df.groupby("action")
             .apply(lambda g: pd.concat([summarize(g,"mse"), summarize(g,"dr_value")]))
             .reset_index(drop=False)
          )

print(summary.sort_values("action"))
summary.to_csv("cknn_vax_actions_summary_nodupp.csv", index=False)


A_dose1
221.857563495636
A_dose2
221.19806933403015
A_dose3
223.66928029060364
A_dose4
222.82362914085388
A_vax_any
223.52686429023743
hello  A_dose1
run   237.82792854309082
hello  A_dose2
run   238.00918054580688
hello  A_dose3
run   237.05921697616577
hello  A_dose4
run   237.35303401947021
hello  A_vax_any


In [3]:
import pandas as pd
X_visit_with_A=pd.read_csv('X_visit_with_A.csv')
pasc_df = pd.read_csv('y_pasc_score2024.csv')
# pasc_df.head()
pasc_df.rename({"pasc_score_2024":"pasc_score"},axis="columns",inplace=True)


In [ ]:
from compare_actions_methods import compare_actions_methods

ACTIONS = ["A_dose1","A_dose2","A_dose3","A_dose4","A_vax_any"]  # (you already ran A_booster)

df_all, summary_all = compare_actions_methods(
    X_visit_df=X_visit_with_A,      # must include PARTICIPANT_ID, VISIT_START_DATE, and all A_* columns + features
    outcome_df=pasc_df,             # or depression_df
    actions=ACTIONS,
    id_col="PARTICIPANT_ID",
    visit_col="VISIT_START_DATE",
    outcome_col="pasc_score",       # or "depression_score"
    outcome_date_col="date",
    outcome_mode="delta_next",      # or "next_value"
    max_forward_days=120,
    seeds=[1,2,3,4,5,6,7,8,9,10],   # paper-ready mean ± 95% CI
    encoder="gru", d_latent=16, hidden=64,
    # encoder="identity",
    k=50, temp=1.0,
    enc_epochs=5, head_epochs=10, batch_size=64, lr=1e-3,
    test_frac=0.2, use_cuda=True,
    standardize_y=False             # set True to z-score Y within each action experiment
)

print(summary_all.head())
# Optional: persist
df_all.to_csv("all_methods_per_seed.csv", index=False)
summary_all.to_csv("all_methods_summary.csv", index=False)
print("done")


In [6]:
print("done")




done
